## Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [81]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [33]:
from keys import key_iex

In [34]:
sp500=pd.read_csv('sp_500_stocks.csv')
sp500

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [35]:
cols=['Ticker', 'Price','Price-to-Earnings Ratio', 'Number Of Shares to Buy']
stock_param=pd.DataFrame(columns=cols )

In [36]:
from create_batches import create_batches

In [37]:
res=list(create_batches(sp500['Ticker'],100))

In [38]:
batch_req=[]
for i in res:
    batch_req.append(','.join(i))
print(batch_req)

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

In [39]:
#updated way
for symbol in batch_req:
    ot=requests.get(f"https://api.iex.cloud/v1/data/core/quote/{symbol}?token={key_iex}").json()
    for cs in ot:
        stock_param.loc[len(stock_param.index)] = [cs['symbol'], 
                                                    cs['latestPrice'], 
                                                    cs['peRatio'], 
                                                    'N/A']

In [40]:
stock_param
krx=stock_param

Now we need to sort this values in increasing ordere with respect to price to earing ratio but we will need to drop the ones with negative peratio because this means that company is in loss which is not a good place to invest money

In [41]:
stock_param.sort_values('Price-to-Earnings Ratio',inplace=True)
stock_param=stock_param[stock_param['Price-to-Earnings Ratio']>0]

In [42]:
best50=stock_param[:50]
best50

,Ticker,Price,Price-to-Earnings Ratio,Number Of Shares to Buy
71,BRK.B,365.980,0.01,N/A
192,FRC,3.510,0.43,N/A
137,DISH,5.770,2.9,N/A
159,EMR,95.290,4.16,N/A
468,VLO,128.395,4.37,N/A
503,ZION,44.225,4.72,N/A
23,ALB,136.260,4.83,N/A
454,UAL,43.455,5.04,N/A
203,GM,36.840,5.19,N/A
1,AAL,14.635,5.46,N/A


In [43]:
best50.reset_index(inplace=True)

In [44]:
best50.drop('index',axis=1,inplace=True)
best50

C:\Users\sjmad\AppData\Local\Temp\ipykernel_20576\464745831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best50.drop('index',axis=1,inplace=True)


,Ticker,Price,Price-to-Earnings Ratio,Number Of Shares to Buy
0,BRK.B,365.980,0.01,N/A
1,FRC,3.510,0.43,N/A
2,DISH,5.770,2.9,N/A
3,EMR,95.290,4.16,N/A
4,VLO,128.395,4.37,N/A
5,ZION,44.225,4.72,N/A
6,ALB,136.260,4.83,N/A
7,UAL,43.455,5.04,N/A
8,GM,36.840,5.19,N/A
9,AAL,14.635,5.46,N/A


Now lets invest 10000000 in the stocks accordingly

In [45]:
su=20*len(best50.index)-best50['Price-to-Earnings Ratio'].sum()
print(su)

631.98


In [46]:
best50['Price-to-Earnings Ratio']=pd.to_numeric(best50['Price-to-Earnings Ratio'])
best50.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   50 non-null     object 
 1   Price                    50 non-null     float64
 2   Price-to-Earnings Ratio  50 non-null     float64
 3   Number Of Shares to Buy  50 non-null     object 
dtypes: float64(2), object(2)
memory usage: 1.7+ KB


C:\Users\sjmad\AppData\Local\Temp\ipykernel_20576\2069741467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best50['Price-to-Earnings Ratio']=pd.to_numeric(best50['Price-to-Earnings Ratio'])


In [47]:
for i in range (0,50):
    best50.loc[i,'Number Of Shares to Buy']=math.floor(((20-best50.loc[i,'Price-to-Earnings Ratio']/su)*10000000)/ best50.loc[i,'Price'])
best50

,Ticker,Price,Price-to-Earnings Ratio,Number Of Shares to Buy
0,BRK.B,365.980,0.01,546477
1,FRC,3.510,0.43,56978118
2,DISH,5.770,2.90,34654092
3,EMR,95.290,4.16,2098165
4,VLO,128.395,4.37,1557154
5,ZION,44.225,4.72,4520640
6,ALB,136.260,4.83,1467221
7,UAL,43.455,5.04,4600627
8,GM,36.840,5.19,5426652
9,AAL,14.635,5.46,13659966


Now since some values are negative in price to earnings ration can be negative for some startups because they are usually in dept in there initial phases so its not he best way to calculate desirability for investment. We will consider following parameters->
* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)


In [48]:
columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

In [49]:
rv_dataframe = pd.DataFrame(columns = columns)

In [51]:
for symbol in batch_req:
    ot=requests.get(f"https://api.iex.cloud/v1/data/CORE/ADVANCED_STATS/{symbol}?token={key_iex}").json()
    sym=symbol.split(',')
    #print(sym)
    for i in range(0,len(ot)):
        cs=ot[i]
        enterprise_value =cs['enterpriseValue']
        ebita=cs["EBITDA"]
        gross_profit=cs["grossProfit"]
        try:
            ev_to_ebitda =enterprise_value/ebita
        except TypeError:
            ev_to_ebitda =np.NaN
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        rv_dataframe.loc[len(rv_dataframe.index)] = [sym[i], 
                                                    krx.loc[len(rv_dataframe.index),'Price'], 
                                                    5,
                                                    krx.loc[len(rv_dataframe.index),'Price-to-Earnings Ratio'], 
                                                     5,
                                                    cs["priceToBook"], 
                                                    5,
                                                    cs["priceToSales"], 
                                                     5,
                                                    ev_to_ebitda,
                                                    5,
                                                    ev_to_gross_profit,
                                                    5,
                                                    5]



In [52]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.490,5,31.62,5,6.54,5,5.6,5,24.305747,5,11.370740,5,5
1,AAL,14.635,5,5.46,5,-1.73,5,0.168,5,4.764746,5,0.600884,5,5
2,AAP,61.790,5,19.8,5,1.38,5,0.324,5,8.322869,5,1.092293,5,5
3,AAPL,184.870,5,30.16,5,45.34,5,7.35,5,22.789545,5,16.951904,5,5
4,ABBV,160.690,5,44.02,5,24.13,5,5.29,5,13.496201,5,9.358414,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.450,5,24.42,5,-4.39,5,5.1,5,19.501980,5,8.848462,5,5
501,ZBH,121.830,5,53.91,5,2,5,3.44,5,15.787066,5,5.804224,5,5
502,ZBRA,262.150,5,29.23,5,4.31,5,2.56,5,17.419792,5,6.436465,5,5
503,ZION,44.225,5,4.72,5,1.34,5,1.31,5,3.593587,5,1.356183,5,5


In [60]:
rv_dataframe.dropna(inplace=True)

In [64]:
rv_dataframe.reset_index(inplace=True)
rv_dataframe.drop('index', axis=1,inplace=True)

In [65]:
rv_dataframe

,level_0,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,0,A,132.490,5,31.62,5,6.54,5,5.6,5,24.305747,5,11.370740,5,5
1,1,AAL,14.635,5,5.46,5,-1.73,5,0.168,5,4.764746,5,0.600884,5,5
2,2,AAP,61.790,5,19.8,5,1.38,5,0.324,5,8.322869,5,1.092293,5,5
3,3,AAPL,184.870,5,30.16,5,45.34,5,7.35,5,22.789545,5,16.951904,5,5
4,4,ABBV,160.690,5,44.02,5,24.13,5,5.29,5,13.496201,5,9.358414,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,479,YUM,128.450,5,24.42,5,-4.39,5,5.1,5,19.501980,5,8.848462,5,5
480,480,ZBH,121.830,5,53.91,5,2,5,3.44,5,15.787066,5,5.804224,5,5
481,481,ZBRA,262.150,5,29.23,5,4.31,5,2.56,5,17.419792,5,6.436465,5,5
482,482,ZION,44.225,5,4.72,5,1.34,5,1.31,5,3.593587,5,1.356183,5,5


In [78]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}
for i in metrics.keys():
    rv_dataframe[i]=pd.to_numeric(rv_dataframe[i])
for i in metrics.values():
    rv_dataframe[i]=pd.to_numeric(rv_dataframe[i])

In [79]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.490,5,31.62,5,6.54,5,5.600,5,24.305747,5,11.370740,5,5
1,AAL,14.635,5,5.46,5,-1.73,5,0.168,5,4.764746,5,0.600884,5,5
2,AAP,61.790,5,19.80,5,1.38,5,0.324,5,8.322869,5,1.092293,5,5
3,AAPL,184.870,5,30.16,5,45.34,5,7.350,5,22.789545,5,16.951904,5,5
4,ABBV,160.690,5,44.02,5,24.13,5,5.290,5,13.496201,5,9.358414,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,YUM,128.450,5,24.42,5,-4.39,5,5.100,5,19.501980,5,8.848462,5,5
480,ZBH,121.830,5,53.91,5,2.00,5,3.440,5,15.787066,5,5.804224,5,5
481,ZBRA,262.150,5,29.23,5,4.31,5,2.560,5,17.419792,5,6.436465,5,5
482,ZION,44.225,5,4.72,5,1.34,5,1.310,5,3.593587,5,1.356183,5,5


In [80]:
rv_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   484 non-null    object 
 1   Price                    484 non-null    float64
 2   Number of Shares to Buy  484 non-null    int64  
 3   Price-to-Earnings Ratio  484 non-null    float64
 4   PE Percentile            484 non-null    int64  
 5   Price-to-Book Ratio      484 non-null    float64
 6   PB Percentile            484 non-null    int64  
 7   Price-to-Sales Ratio     484 non-null    float64
 8   PS Percentile            484 non-null    int64  
 9   EV/EBITDA                484 non-null    float64
 10  EV/EBITDA Percentile     484 non-null    int64  
 11  EV/GP                    484 non-null    float64
 12  EV/GP Percentile         484 non-null    int64  
 13  RV Score                 484 non-null    int64  
dtypes: float64(6), int64(7), o

In [82]:


for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

In [83]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.490,5,31.62,0.741736,6.54,0.776860,5.600,0.805785,24.305747,0.865702,11.370740,0.814050,5
1,AAL,14.635,5,5.46,0.097107,-1.73,0.066116,0.168,0.012397,4.764746,0.053719,0.600884,0.018595,5
2,AAP,61.790,5,19.80,0.475207,1.38,0.188017,0.324,0.043388,8.322869,0.210744,1.092293,0.057851,5
3,AAPL,184.870,5,30.16,0.708678,45.34,0.975207,7.350,0.876033,22.789545,0.842975,16.951904,0.935950,5
4,ABBV,160.690,5,44.02,0.855372,24.13,0.960744,5.290,0.789256,13.496201,0.481405,9.358414,0.700413,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,YUM,128.450,5,24.42,0.559917,-4.39,0.061983,5.100,0.780992,19.501980,0.762397,8.848462,0.669421,5
480,ZBH,121.830,5,53.91,0.900826,2.00,0.332645,3.440,0.625000,15.787066,0.584711,5.804224,0.433884,5
481,ZBRA,262.150,5,29.23,0.696281,4.31,0.644628,2.560,0.541322,17.419792,0.673554,6.436465,0.489669,5
482,ZION,44.225,5,4.72,0.088843,1.34,0.175620,1.310,0.251033,3.593587,0.024793,1.356183,0.092975,5


Now lets calculate ev score as mean of each of these parameters

In [89]:
for i in range (0,len(rv_dataframe.index)):
    temp=[]
    for j in metrics.values():
        temp.append(rv_dataframe.loc[i,j])
    rv_dataframe.loc[i,'RV Score']=np.mean(temp)

In [90]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.490,5.0,31.62,0.741736,6.54,0.776860,5.600,0.805785,24.305747,0.865702,11.370740,0.814050,0.800826
1,AAL,14.635,5.0,5.46,0.097107,-1.73,0.066116,0.168,0.012397,4.764746,0.053719,0.600884,0.018595,0.049587
2,AAP,61.790,5.0,19.80,0.475207,1.38,0.188017,0.324,0.043388,8.322869,0.210744,1.092293,0.057851,0.195041
3,AAPL,184.870,5.0,30.16,0.708678,45.34,0.975207,7.350,0.876033,22.789545,0.842975,16.951904,0.935950,0.867769
4,ABBV,160.690,5.0,44.02,0.855372,24.13,0.960744,5.290,0.789256,13.496201,0.481405,9.358414,0.700413,0.757438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,YUM,128.450,5.0,24.42,0.559917,-4.39,0.061983,5.100,0.780992,19.501980,0.762397,8.848462,0.669421,0.566942
480,ZBH,121.830,5.0,53.91,0.900826,2.00,0.332645,3.440,0.625000,15.787066,0.584711,5.804224,0.433884,0.575413
481,ZBRA,262.150,5.0,29.23,0.696281,4.31,0.644628,2.560,0.541322,17.419792,0.673554,6.436465,0.489669,0.609091
482,ZION,44.225,5.0,4.72,0.088843,1.34,0.175620,1.310,0.251033,3.593587,0.024793,1.356183,0.092975,0.126653


In [91]:
rv_dataframe.sort_values('RV Score',inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
180,FRC,3.510,5.0,0.43,0.082645,0.0456,0.068182,0.0934,0.002066,0.270279,0.012397,0.094878,0.006198,0.034298
1,AAL,14.635,5.0,5.46,0.097107,-1.7300,0.066116,0.1680,0.012397,4.764746,0.053719,0.600884,0.018595,0.049587
190,GM,36.840,5.0,5.19,0.095041,0.6618,0.072314,0.2866,0.028926,1.333389,0.014463,0.941775,0.041322,0.050413
164,F,11.995,5.0,7.84,0.119835,1.0500,0.113636,0.2674,0.022727,2.382694,0.016529,1.398197,0.103306,0.075207
434,UAL,43.455,5.0,5.04,0.092975,1.5500,0.230372,0.2610,0.020661,3.268258,0.022727,0.505515,0.012397,0.075826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,CDNS,260.580,5.0,74.24,0.952479,22.0800,0.956612,17.5300,0.993802,59.228559,0.981405,19.571990,0.962810,0.969421
322,NOW,696.430,5.0,90.09,0.960744,19.3000,0.938017,16.3600,0.985537,115.969991,0.993802,20.443763,0.971074,0.969835
141,DXCM,129.620,5.0,147.04,0.981405,21.1200,0.948347,14.0800,0.975207,91.586285,0.991736,21.750479,0.981405,0.975620
328,NVDA,517.610,5.0,68.20,0.940083,36.4600,0.973140,27.0300,1.000000,54.337586,0.977273,38.417538,1.000000,0.978099


In [92]:
rv_dataframe.reset_index(inplace=True)
rv_dataframe.drop('index', axis=1,inplace=True)

In [93]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.510,5.0,0.43,0.082645,0.0456,0.068182,0.0934,0.002066,0.270279,0.012397,0.094878,0.006198,0.034298
1,AAL,14.635,5.0,5.46,0.097107,-1.7300,0.066116,0.1680,0.012397,4.764746,0.053719,0.600884,0.018595,0.049587
2,GM,36.840,5.0,5.19,0.095041,0.6618,0.072314,0.2866,0.028926,1.333389,0.014463,0.941775,0.041322,0.050413
3,F,11.995,5.0,7.84,0.119835,1.0500,0.113636,0.2674,0.022727,2.382694,0.016529,1.398197,0.103306,0.075207
4,UAL,43.455,5.0,5.04,0.092975,1.5500,0.230372,0.2610,0.020661,3.268258,0.022727,0.505515,0.012397,0.075826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,CDNS,260.580,5.0,74.24,0.952479,22.0800,0.956612,17.5300,0.993802,59.228559,0.981405,19.571990,0.962810,0.969421
480,NOW,696.430,5.0,90.09,0.960744,19.3000,0.938017,16.3600,0.985537,115.969991,0.993802,20.443763,0.971074,0.969835
481,DXCM,129.620,5.0,147.04,0.981405,21.1200,0.948347,14.0800,0.975207,91.586285,0.991736,21.750479,0.981405,0.975620
482,NVDA,517.610,5.0,68.20,0.940083,36.4600,0.973140,27.0300,1.000000,54.337586,0.977273,38.417538,1.000000,0.978099


In [94]:
rv_dataframe=rv_dataframe[:50]

lets distribute 10000000 in these stocks

In [98]:
su=10000000/len(rv_dataframe.index)
for i in range (0,50):
    rv_dataframe.loc[i,'Number of Shares to Buy']=math.floor(su/rv_dataframe.loc[i,'Price'])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.510,56980.0,0.43,0.082645,0.0456,0.068182,0.0934,0.002066,0.270279,0.012397,0.094878,0.006198,0.034298
1,AAL,14.635,13665.0,5.46,0.097107,-1.7300,0.066116,0.1680,0.012397,4.764746,0.053719,0.600884,0.018595,0.049587
2,GM,36.840,5428.0,5.19,0.095041,0.6618,0.072314,0.2866,0.028926,1.333389,0.014463,0.941775,0.041322,0.050413
3,F,11.995,16673.0,7.84,0.119835,1.0500,0.113636,0.2674,0.022727,2.382694,0.016529,1.398197,0.103306,0.075207
4,UAL,43.455,4602.0,5.04,0.092975,1.5500,0.230372,0.2610,0.020661,3.268258,0.022727,0.505515,0.012397,0.075826
5,UNM,45.380,4407.0,7.32,0.111570,0.9296,0.092975,0.7301,0.113636,4.314061,0.037190,0.730077,0.028926,0.076860
6,C,53.720,3723.0,8.57,0.146694,0.5472,0.070248,0.7038,0.105372,4.583699,0.047521,0.740865,0.030992,0.080165
7,KSS,27.135,7370.0,-20.87,0.035124,0.7804,0.082645,0.1669,0.010331,9.008233,0.229339,1.224908,0.080579,0.087603
8,SYF,39.135,5110.0,7.25,0.109504,1.2300,0.153926,0.7516,0.115702,4.528865,0.045455,1.003240,0.045455,0.094008
9,VIAC,29.580,6761.0,5.75,0.099174,0.9396,0.095041,0.7205,0.109504,6.649893,0.111570,1.201202,0.078512,0.098760


In [99]:
# rv_dataframe.drop('Number Of Shares to Buy',axis=1,inplace=True)
# rv_dataframe
rv_dataframe['Number of Shares to Buy']=rv_dataframe['Number of Shares to Buy'].astype(int)
rv_dataframe

C:\Users\sjmad\AppData\Local\Temp\ipykernel_20576\1988340476.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe['Number of Shares to Buy']=rv_dataframe['Number of Shares to Buy'].astype(int)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.510,56980,0.43,0.082645,0.0456,0.068182,0.0934,0.002066,0.270279,0.012397,0.094878,0.006198,0.034298
1,AAL,14.635,13665,5.46,0.097107,-1.7300,0.066116,0.1680,0.012397,4.764746,0.053719,0.600884,0.018595,0.049587
2,GM,36.840,5428,5.19,0.095041,0.6618,0.072314,0.2866,0.028926,1.333389,0.014463,0.941775,0.041322,0.050413
3,F,11.995,16673,7.84,0.119835,1.0500,0.113636,0.2674,0.022727,2.382694,0.016529,1.398197,0.103306,0.075207
4,UAL,43.455,4602,5.04,0.092975,1.5500,0.230372,0.2610,0.020661,3.268258,0.022727,0.505515,0.012397,0.075826
5,UNM,45.380,4407,7.32,0.111570,0.9296,0.092975,0.7301,0.113636,4.314061,0.037190,0.730077,0.028926,0.076860
6,C,53.720,3723,8.57,0.146694,0.5472,0.070248,0.7038,0.105372,4.583699,0.047521,0.740865,0.030992,0.080165
7,KSS,27.135,7370,-20.87,0.035124,0.7804,0.082645,0.1669,0.010331,9.008233,0.229339,1.224908,0.080579,0.087603
8,SYF,39.135,5110,7.25,0.109504,1.2300,0.153926,0.7516,0.115702,4.528865,0.045455,1.003240,0.045455,0.094008
9,VIAC,29.580,6761,5.75,0.099174,0.9396,0.095041,0.7205,0.109504,6.649893,0.111570,1.201202,0.078512,0.098760


Lets save it to excel

In [100]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [101]:
dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
           
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0000',
           
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.00%',
            
        }
    )

In [102]:
column_formats = {
                   
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [103]:
writer.save()

C:\Users\sjmad\AppData\Local\Temp\ipykernel_20576\934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
